In [2]:
import numpy as np
import sympy as sp

In [71]:
class StringGeneration():
    def random_problem(self, kind, size):
        if kind == 'dice':
            target = ''.join(np.random.choice(['1','2','3','4','5','6'],size=size))
        elif kind == 'coin':
            target = ''.join(np.random.choice(['H','T'],size=size))
            
        print(f'\nWhat is the expected number of steps to get the sequence {target} with a fair {kind}?')
        self.solve(kind, target)
        
    def solve(self, kind, target):
        '''
        Solve using Absorbing Markov Chain
        https://en.wikipedia.org/wiki/Absorbing_Markov_chain
        
        TO DO:
        - Add option for a biased dice/coin
        
        '''
        if kind == 'dice':
            outcome_probs = {c:sp.Rational('1/6') for c in ['1','2','3','4','5','6']}
        elif kind == 'coin':
            outcome_probs = {c:sp.Rational('1/2') for c in ['H','T']}
        states = [target[:i] for i in range(len(target)+1)]
        num_states = len(states)

        # construct transition matrix
        transition_matrix = []
        # starting from transient states
        for start_string in states[:-1]:
            probs = [0]*num_states
            for outcome, prob in outcome_probs.items():
                current_string = start_string + outcome
                state_index = 0
                for i in range(len(current_string)):
                    if current_string[i:] in states:
                        state_index = states.index(current_string[i:])
                        break
                probs[state_index] += prob
            transition_matrix.append(probs)
        # final row for absorbing state
        probs = [0]*num_states
        probs[-1] = 1
        transition_matrix.append(probs)

        # compute transition matrix and the number of expected steps
        P = sp.Matrix(transition_matrix)
        N = (sp.eye(num_states-1) - P[:-1,:-1]).inv()
        expected_num_steps = N * sp.ones(N.shape[1], 1)
        print(f'Expected # of steps to get {target}:',expected_num_steps.flat(),'starting from',states[:-1])

In [72]:
SG = StringGeneration()
SG.solve('coin','HHH')
SG.solve('dice','66')
SG.solve('coin','HTH')
SG.random_problem('coin',2)

Expected # of steps to get HHH: [14, 12, 8] starting from ['', 'H', 'HH']
Expected # of steps to get 66: [42, 36] starting from ['', '6']
Expected # of steps to get HTH: [10, 8, 6] starting from ['', 'H', 'HT']

What is the expected number of steps to get the sequence TH with a fair coin?
Expected # of steps to get TH: [4, 2] starting from ['', 'T']
